In [4]:
import pandas as pd
from datetime import datetime, timedelta
import os

# The reason from and to gtfs generation dates are included is to ensure that different
# timeframes' gtfs service_id's trip_ids are maintained

In [5]:
# Retrieve trips.txt files for each gtfs generation dates
extract_path = "./files/extracted/"
trips_path = "/trips.txt"
gtfs_generation_dates = [item for item in os.listdir(extract_path) if os.path.isdir(os.path.join(extract_path, item))]

In [6]:
def process_trips_dates_file(current_gtfs_date, next_gtfs_date):
    # Read the trips.txt file into a DataFrame
    transfers = pd.read_csv(extract_path + current_gtfs_date + trips_path)
    transfers['current_gtfs_date'] = current_gtfs_date
    transfers['next_gtfs_date'] = next_gtfs_date
    # transfers['next_gtfs_date'] = 99991231 if next_gtfs_date == None else next_gtfs_date

    return(transfers)

In [7]:
# total_trips = pd.DataFrame()
# # Set the path to the GTFS files
# for gtfs_date in gtfs_generation_dates:
#     # calendar_file = folder_path + filename # example: calendar_20181221.txt'
#     total_trips = pd.concat([total_trips, process_trips_dates_file(gtfs_date)])

# # Group by trip_id and service_id and get the minimum value_amount in each group
# grouped_df = total_trips.groupby(["trip_id", "service_id"]).agg({"gtfs_generation_date": "min"}).reset_index()

In [8]:
total_trips = pd.DataFrame()
for i, item in enumerate(gtfs_generation_dates):
    # Set the last date for 'next_item' to be highdate
    next_item = gtfs_generation_dates[i+1] if i < len(gtfs_generation_dates)-1 else 99991231
    total_trips = pd.concat([total_trips, process_trips_dates_file(item, next_item)])

In [9]:
# Clean total_transfers down to columns that are applicable for the logic going forward
total_transfers = total_trips[['service_id','trip_id','current_gtfs_date','next_gtfs_date']]
unique_transfers = total_transfers.drop_duplicates()

In [10]:
# Generate a mappings folder if it doesn't exist
if not os.path.exists('mappings'):
    os.makedirs('mappings')

# Generate a mapping file for future reference
unique_transfers.to_csv('mappings/services_to_trips.csv', index=False)